In [1]:
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from thefuzz import fuzz

c:\Users\Joshualevy\Anaconda3\envs\ioCapture\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def load_input_csvs(pub_code):
    print('READING IN {}'.format(pub_code))
    scopus_core_path = 'scopus_data/' + pub_code + '_scopus_core.csv'
    econlit_path = 'econlit_xml_csv/' + pub_code + '_econlit.csv'

    scopus_df = pd.read_csv(scopus_core_path, encoding='utf-8')
    econlit_df = pd.read_csv(econlit_path, encoding='utf-8')

    print('{} ------ NUMBER OF SCOPUS OBSERVATIONS: {}'.format(pub_code, len(scopus_df)))
    print('{} ----- NUMBER OF ECONLIT OBSERVATIONS: {}'.format(pub_code, len(econlit_df)))


    return scopus_df, econlit_df

In [72]:
non_article_patterns = {
    'addresses' : r'President\'s address|Presidential Address|Keynote address', 
    'abstracts' : r'^Abstracts', 
    'announcements' : r'Editorial Announcement|^Announcements?(\.|:)',
    'acknowledgements' : r'Acknowledge?ments\.',
    'backmatter' : r'Back Cover|Back matter|Backmatter',
    'corrigendum': r'Corrigendum( to)?:?',
    'comment' : r':( A)? Comment|Comments? on|Comment:| Comment$|(: )?Notes and comments(: )?|Introductory comments:|^Comment|--Comment',
    'contents' : r'Table of contents?\.|^Contents',
    'correction' : r':( A)? Correction|Correction to',
    'cover' : r'^Cover(\.)?$',
    'discussion' : r':( Panel)? discussion|^Discussion\.?$|Discussion of(:| \"| \')|Discussion:| Discussion$',
    'editorial' : r'^Editorial$|(:|Joint) Editorial|Editorial board|Letter from the( new)? editor|Editorial (collaborators|data|:)|Editors\' introduction|^Editorial\.?|Guest editor\'s introduction$',
    'errata': r'Errat(a|um):?|Errata( and corrections)?',
    'forthcoming' : r'forthcoming papers|forthcoming articles',
    'frontmatter' : r'Front Matter|Frontmatter|Preface|Masthead|^Introduction\.?$|Foreword|Front cover\.?',
    'indices' : r'Index(\.)',
    'issue' : r'Issue Information\.',
    'manuscripts' : r'(accepted|forthcoming) manuscripts',
    'meeting' : r'annual meeting',
    'memorial' : r'In memoriam:',
    'minutes' : r'Minutes of the',
    'notes' : r'Editors\' notes?|(A )?Note from the editor|Editor\'s note|Introductory Note|Editor’s Note',
    'proceedings' : r'Papers and Proceedings of|Proceedings of the ',
    'referees': r'Referees\.',
    'rejoinder': r'(: )?Rejoinder:?|Rejoinder to',
    'reply': r'Reply( to)?:|Response:|: Reply|Reply \[|A reply$',
    'response' : r':( A)? Response|A Response \[|^response to',
    'reporting' : r'Recent Referees\.|Report of the ((executive )?secretary|treasurer|editor|president|(search |finance )?committee|director|representative)|List of online reports|(The )?Annual report of the (society)|Independent Auditor(\'s|s\') report',
    'retraction' : r'Retracted:',
    'reviews' : r'Book reviews?',
    'subscription' : r'Subscription Page',
    'symposium' : r'Symposium:',
    'untitled' : r'Untitled\.',
    'others' : r'Nobel lecture|\(Book|Book review\.|News notes\.|Invited papers and discussions|Job openings for',
    'aer_specific' : r':? Distinguished Fellow|Ad hoc Committee|American Economic Review|American Economic Journal:|American Economic Association|Journal of Economic (Perspectives|Literature)|Committee on (Economic Education|Government Relations|the Status of (Women|Minority Groups))|John Bates Clark (Award|Medalist)|Excellence in Refereeing Award|Report of the AEA Committee',
    'eca_specific' : r'Econometric Society|Submission of Manuscripts to Econometrica|Nomination of fellows',
    'jpe_specific' : r'JPE Turnaround Times(, Previous Two Years)?|Index to Vo',
    'qje_specific' : r'The Quarterly Journal of Economics',
    'res_specific' : r'Review of Economic Studies',
    'jem_specific' : r'Issue information - |Introduction to Volume',
    'jhr_specific' : r'Social Science Research and Polocy',
    'rje_specific' : r'RAND Journal of Economics: Introduction',
    'jof_specific' : r'American Finance Association',
    'jol_specific' : r'Sherwin Rozen Prize|Gregg Lewis Prize|Jacob Mincer Prize|SOLE Prize|Officers of the Society',

}

In [4]:
def filter_non_articles(pub_code, scopus_df, econlit_df, print_mode="none"):

    scopus_df['non_article_indicator'] = scopus_df['sc_title'].apply(lambda x: pattern_matching(x))
    econlit_df['non_article_indicator'] = econlit_df['title'].apply(lambda x: pattern_matching(x))

    scopus_non_articles_df = scopus_df[scopus_df.non_article_indicator ==1]
    scopus_non_articles_titles = len(scopus_non_articles_df)
    econlit_non_articles_df = econlit_df[econlit_df.non_article_indicator ==1]
    econlit_non_articles_titles = len(econlit_non_articles_df)


    print('{} ------ SCOPUS NON-ARTICLES REMOVED: {}'.format(pub_code, scopus_non_articles_titles))
    print('{} ----- ECONLIT NON-ARTICLES REMOVED: {}'.format(pub_code, econlit_non_articles_titles))


    if print_mode == "remaining":
        # DEBUGGING/EDITING: Print on all articles *except* those that are filtered out
        for title in scopus_df[scopus_df.non_article_indicator ==0]['sc_title'].tolist():
            print(title)
        print('-------------------------------------------------------------------')
        for title in econlit_df[econlit_df.non_article_indicator==0]['title'].tolist():
            print(title)
    elif print_mode == "removed":
        # DEBUGGING/EDITING: Print *only* articles that are filtered out
        for title in scopus_df[scopus_df.non_article_indicator == 1]['sc_title'].tolist():
            print(title)
        print('-------------------------------------------------------------------')
        for title in econlit_df[econlit_df.non_article_indicator == 1]['title'].tolist():
            print(title)
        


    scopus_df = scopus_df[scopus_df.non_article_indicator == 0]
    econlit_df = econlit_df[econlit_df.non_article_indicator ==0]



    return scopus_df, econlit_df

def pattern_matching(title_string):
    for pattern in non_article_patterns.values():
        if re.search(pattern, title_string, re.I):
            return 1
    return 0


In [5]:
def naive_match(scopus_df, econlit_df):
    scopus_df['sc_title_upper'] = scopus_df['sc_title'].apply(lambda x: x.upper())
    econlit_df['title_upper'] = econlit_df['title'].apply(lambda x: x.upper())

    try:
        naive_match_df = pd.merge(scopus_df, econlit_df,
            how='outer',
            left_on=['sc_vol', 'sc_issue', 'sc_title_upper'],
            right_on=['volume', 'issue', 'title_upper'],
            indicator=True)

    except ValueError:
        print('Initial naive match failed.\nTrying naive match again with coerced column-types')
        scopus_df = scopus_df.astype({
            'sc_title_upper' : 'object',
            'sc_vol' : 'str',
            'sc_issue' : 'str'
        })

        econlit_df = econlit_df.astype({
            'title_upper' : 'object',
            'volume' : 'str',
            'issue' : 'str'
        })

        naive_match_df = pd.merge(scopus_df, econlit_df,
            how='outer',
            left_on=['sc_vol', 'sc_issue', 'sc_title_upper'], 
            right_on=['volume', 'issue', 'title_upper'],
            indicator=True)
        

    ### If the initial merge returns absolutely nothing (probably because issues are incorrect, we try a more permissive naive match)


    return naive_match_df

In [6]:
def left_right_onlys(naive_match_df):
    nm_match_count = len(naive_match_df[naive_match_df._merge == 'both'])
    nm_scopus_only = naive_match_df[naive_match_df._merge == 'left_only']
    nm_econlit_only = naive_match_df[naive_match_df._merge == 'right_only']

    nm_scopus_only.reset_index(inplace=True)
    nm_econlit_only.reset_index(inplace=True)

    nm_scopus_only = nm_scopus_only.drop(columns=['_merge', 'level_0'], axis=1)
    nm_econlit_only = nm_econlit_only.drop(columns=['_merge', 'level_0'], axis=1)

    print('Number of naively-matched observations: {}'.format(nm_match_count))
    print('Number of SCOPUS-only observations: {}'.format(len(nm_scopus_only)))
    print('Number of ECONLIT-only observations: {}'.format(len(nm_econlit_only)))

    return nm_scopus_only, nm_econlit_only

In [7]:
def custom_score_compute(econlit_row, scopus_volume, scopus_issue, scopus_pagerange, scopus_doi, scopus_title_upper, scopus_date):
    econlit_volume = econlit_row.volume
    econlit_issue = econlit_row.issue
    econlit_doi = econlit_row.doi_y
    econlit_pagerange = econlit_row.pages 
    econlit_title_upper = econlit_row.title_upper
    econlit_date = econlit_row.date

    scopus_volume_type = type(scopus_volume)
    scopus_issue_type = type(scopus_issue)
    econlit_volume_type = type(econlit_volume)
    econlit_issue_type = type(econlit_issue)
    type_list = [scopus_volume_type, scopus_issue_type, econlit_volume_type, econlit_issue_type]

    number_types_set = {np.float64, float, int}
    
    # If everything already is of a numerical type, skip the below because we can just coerce everything to an int() and efficiently check for matches that way
    exception_status = 0
    if not set(type_list).issubset(number_types_set):
        for i, element in enumerate(type_list):
            if element == str:
                if i == 0:
                    try:
                        scopus_volume = int(scopus_volume)
                        scopus_volume_type = int
                    except:
                        # print("COULD NOT COERCE SCOPUS VOLUME ({}) PROPERLY".format(scopus_volume))
                        exception_status += 1
                        if scopus_volume == 'SCOPUS FAILURE':
                            exception_status -= 1
                        scopus_volume = -888888
                elif i == 1:
                    try:
                        scopus_issue = int(scopus_issue)
                        scopus_issue_type = int
                    except:
                        # print("COULD NOT COERCE SCOPUS ISSUE ({}) PROPERLY".format(scopus_issue))
                        exception_status += 1
                        if scopus_issue == 'SCOPUS FAILURE':
                            exception_status -= 1
                        scopus_issue = -888888
                elif i == 2:
                    try:
                        econlit_volume = int(econlit_volume)
                        econlit_volume_type = int
                    except:
                        # print("COULD NOT COERCE ECONLIT VOLUME ({}) PROPERLY".format(econlit_volume))
                        exception_status += 1
                        if econlit_volume == 'ECONLIT None Found':
                            exception_status -= 1
                        econlit_volume = -999999
                elif i == 3:
                    try:
                        econlit_issue = int(econlit_issue)
                        econlit_issue_type = int
                    except:
                        # print("COULD NOT COERCE ECONLIT ISSUE ({}) PROPERLY".format(econlit_issue))
                        exception_status += 1
                        if econlit_issue == 'ECONLIT None Found':
                            exception_status -= 1
                        econlit_issue = -999999
        # if exception_status == 1:
        #     return 0



    score = 0
    ### 10 points for being in the correct volume, issue
    if (int(scopus_volume), int(scopus_issue)) == (int(econlit_volume), int(econlit_issue)):
        score += 10
    ### 7 points for being in the at least the correct volume
    elif int(scopus_volume) == int(econlit_volume):
        score += 7
    ### 2 points for being in at least the same issue number
    elif (int(scopus_issue) == int(econlit_issue)) :
        score += 2
    else:
        score = 0
        return score
    ### 10 points for having *exactly* the same page range
    if scopus_pagerange == econlit_pagerange:
        score += 10

    ### 7 points for the publication dates being *exactly* the same (This should be identical to the (Vol, iss) == (Vol, iss) condition)
    if scopus_date == econlit_date:
        score +=7



    ### Up to 10 points for Title fuzzy-match edit-distance
    set_edit_distance_ratio = fuzz.token_set_ratio(econlit_title_upper, scopus_title_upper)
    sort_edit_distance_ratio = fuzz.token_sort_ratio(econlit_title_upper, scopus_title_upper)
    gen_edit_distance_ratio = fuzz.ratio(econlit_title_upper, scopus_title_upper)
    
    # Each of the ratios returns a score on the interval [0,100] so the average will also be on this interval
    average_edit_distance_ratio = np.mean([set_edit_distance_ratio, sort_edit_distance_ratio, gen_edit_distance_ratio])
    
    # print('\t\u251d{}\n\t|\t\t\u251d-AVERAGE SCORE: {}\n\t|\t\t\u251d-set score: {}\n\t|\t\t\u251d-sort score: {}\n\t|\t\t\u2517- gen score: {}'.format(econlit_title_upper, average_edit_distance_ratio, set_edit_distance_ratio, sort_edit_distance_ratio, gen_edit_distance_ratio))


    # Divide the [0,100] by 10 to rescale to [0,10] points towards the custom score
    score += average_edit_distance_ratio/10



    # if exception_status >0 :
    #     print('({},{}), ({},{})'.format(scopus_volume, scopus_issue, econlit_volume, econlit_issue))
    #     return 0



    return score

def custom_scorer(scopus_row, econlit_only_df):
    scopus_volume = scopus_row.sc_vol
    scopus_issue = scopus_row.sc_issue
    scopus_pagerange = scopus_row.sc_page_range
    scopus_doi = scopus_row.doi_x
    scopus_title_upper = scopus_row.sc_title_upper
    scopus_date = scopus_row.sc_pub_date


    econlit_only_df.loc[:, 'custom_match_score'] = econlit_only_df.apply(lambda x: custom_score_compute(x, scopus_volume, scopus_issue, scopus_pagerange, scopus_doi, scopus_title_upper, scopus_date), axis=1)

    econlit_only_df_scored_list = econlit_only_df['custom_match_score'].tolist()
    return econlit_only_df_scored_list


In [8]:
def index_matching(score_matrix):
    rows = score_matrix.shape[0]
    cols = score_matrix.shape[1]
    print("SCORE MATRIX DIMENSIONS: ({} ROWS, {} COLS)".format(rows, cols))

    matched_pairs = []

    # rows
    for scopus_index in range(0, rows):
        # First we find the column (econlit article) that has the highest matching-score for this scopus observation
        best_match_score = max(score_matrix[scopus_index])      
        best_match_index = np.argmax(score_matrix[scopus_index])


        # Then we make sure that the best match for this scopus article does not match better with another scopus article

        best_matchs_match_index = np.argmax(score_matrix[:, best_match_index], axis=0)
        best_matchs_match_score = score_matrix[best_matchs_match_index][best_match_index]

    
        if scopus_index == best_matchs_match_index:
            ##### THIS IS THE KEY VALUE THAT WE USE TO DETERMINE THE THRESHOLD FOR WHETHER A FUZZY MATCH IS SUFFICIENTLY GOOD
            if best_match_score < 19.9:
                continue
            else:
                matched_pairs.append((int(scopus_index), int(best_match_index)))

        else:
            print('On Scopus index {}, the best match appears to be Econlit index {} (match score: {}), but that Econlit matches best with Scopus index {} (match score: {})'.format(scopus_index, best_match_index, best_match_score, best_matchs_match_index, best_matchs_match_score))

    return matched_pairs
        


In [9]:
def interpret_matches(matched_pairs, nm_scopus_only, nm_econlit_only):
    print('Interpreting {} matches'.format(len(matched_pairs)))

    for pair in matched_pairs:
        scopus_index = pair[0]
        econlit_index = pair[1]

        scopus_title = nm_scopus_only.loc[scopus_index, 'sc_title_upper']
        econlit_title = nm_econlit_only.loc[econlit_index, 'title_upper']

        match_score = unmatched_score_matrix[scopus_index][econlit_index]

        print('Based on scores({}), want to match (({})) with (({}))'.format(match_score, scopus_title, econlit_title))



In [10]:
def matrix_instantiation(scopus_only, econlit_only):
    # Rows (number of scopus-only articles)
    n = len(scopus_only)
    # Columns (number of econlit-only articles)
    m = len(econlit_only)

    matrix = np.zeros((n,m))

    for i in range(0,n):
        scopus_row = scopus_only.iloc[i, :]
        temp_econlit_df = econlit_only
        print(scopus_row.sc_title_upper)
        matrix[i][0:m] = custom_scorer(scopus_row, temp_econlit_df)
        print('\n\n')

    return matrix

In [11]:
def append_fuzzy_matches(fuzzy_matches_indices, scopus_only, econlit_only, naive_matched_df):
    
    scopus_columns = ['doi_x', 'sc_title', 'sc_issn', 'sc_pub_name', 'sc_vol', 'sc_issue', 'sc_page_range', 'sc_abstract_api_endpoint', 'sc_human_url', 'sc_pub_date', 'sc_open_access_status', 'sc_query_used', 'sc_title_upper']
    econlit_columns = ['jel_desc', 'jel_code', 'doi_y', 'title', 'volume', 'issue', 'date', 'pages', 'issn', 'author', 'abstract', 'title_upper']
    
    matching_columns = scopus_columns + econlit_columns

    
    fuzzy_matched_df = pd.DataFrame(np.full((0, len(matching_columns)), np.nan), columns=matching_columns)

    for i, pair in enumerate(fuzzy_matches_indices):
        scopus_index = pair[0]
        econlit_index = pair[1]

        fuzzy_matched_df.loc[i, scopus_columns] = scopus_only.loc[scopus_index, scopus_columns]
        fuzzy_matched_df.loc[i, econlit_columns] = econlit_only.loc[econlit_index, econlit_columns]


    fuzzy_matches_appended = pd.concat([naive_matched_df, fuzzy_matched_df], ignore_index=True)
    return fuzzy_matches_appended

In [12]:
def fuzzy_unmatched_remainders(matched_indices, nm_scopus_only_df, nm_econlit_only_df):
    
    fuzzy_unmatched_scopus = nm_scopus_only_df
    fuzzy_unmatched_econlit = nm_econlit_only_df

    for pair in matched_indices:
        matched_scopus_index = pair[0]
        matched_econlit_index = pair[1]

        fuzzy_unmatched_scopus = fuzzy_unmatched_scopus.drop([matched_scopus_index])
        fuzzy_unmatched_econlit = fuzzy_unmatched_econlit.drop([matched_econlit_index])


    return fuzzy_unmatched_scopus, fuzzy_unmatched_econlit

In [13]:
def output_fuzzy_matched_etc(pub_code: str, fuzzy_matches_appended: pd.DataFrame, fuzzy_unmatched_scopus: pd.DataFrame, fuzzy_unmatched_econlit: pd.DataFrame):
    fuzzy_folder_path = 'econlit_scopus_matching_out/{}_fuzzy_results/'.format(pub_code)
    fuzzy_matches_appended_path = fuzzy_folder_path + '{}_fuzzy_matches.csv'.format(pub_code)
    unmatched_scopus_path = fuzzy_folder_path + '{}_fuzzy_unmatched_scopus.csv'.format(pub_code)
    unmatched_econlit_path = fuzzy_folder_path + '{}_fuzzy_unmatched_econlit.csv'.format(pub_code)

    if os.path.exists(fuzzy_folder_path):
        fuzzy_matches_appended.to_csv(fuzzy_matches_appended_path, encoding='utf-8', index=False)
        fuzzy_unmatched_scopus.to_csv(unmatched_scopus_path, encoding='utf-8', index=False)
        fuzzy_unmatched_econlit.to_csv(unmatched_econlit_path, encoding='utf-8', index=False)


    else:
        print('{} folder/path does not exist'.format(fuzzy_folder_path))
        print('Creating path now')
        os.makedirs(fuzzy_folder_path)
        output_fuzzy_matched_etc(pub_code, fuzzy_matches_appended, fuzzy_unmatched_scopus, fuzzy_unmatched_econlit)
    return

In [14]:
def generate_matching_report(pub_code, scopus_df, econlit_df, naive_match_df, nm_scopus_only, nm_econlit_only, fuzzy_matches_appended, fuzzy_unmatched_scopus, fuzzy_unmatched_econlit):

    bars = '----------------------------------------------------------------------'

    # ORIGINALS
    scopus_original = 'Original {} Scopus-collected observations: {}'.format(pub_code, len(scopus_df))
    econlit_original = 'Original {} EconLit-collected observations: {}'.format(pub_code, len(econlit_df))
    total_original = 'Original {} collection TOTAL observations: {}'.format(pub_code, len(scopus_df) + len(econlit_df))


    # POST-NAIVE-MATCH
    naive_match = 'Number of {} naively-matched observations: {}'.format(pub_code, len(naive_match_df))
    post_naive_scopus_only = 'Post-naive match {} Scopus-only observations: {}'.format(pub_code, len(nm_scopus_only))
    post_naive_econlit_only = 'Post-naive match {} EconLit-only observations: {}'.format(pub_code, len(nm_econlit_only))


    ### POST-FUZZY-MATCH
    fuzzy_matches_added = len(fuzzy_matches_appended) - len(naive_match_df)
    fuzzy_match = 'Number of {} fuzzy-matched observations {} ({} added)'.format(pub_code, len(fuzzy_matches_appended), fuzzy_matches_added)
    post_fuzzy_scopus_only = 'Number of {} post-fuzzy-match Scopus-only remainder observations: {}'.format(pub_code, len(fuzzy_unmatched_scopus))
    post_fuzzy_econlit_only = 'Number of {} post-fuzzy-match EconLit-only remainder observations: {}'.format(pub_code, len(fuzzy_unmatched_econlit))
    total_fuzzy_unmatched = len(fuzzy_unmatched_scopus) + len(fuzzy_unmatched_econlit)

    fuzzy_unmatched_summary = '{} observations ({}% of {}) remain unmatched.'.format(total_fuzzy_unmatched, round((total_fuzzy_unmatched * 100 )/ len(fuzzy_matches_appended), 2), len(fuzzy_matches_appended))


    try: 
        fuzzy_unmatched_scopus['titles_plus_plus'] = fuzzy_unmatched_scopus.apply(lambda obs: '{} (VOL. {} ({}))'.format(obs.sc_title, int(obs.sc_vol), int(obs.sc_issue)), axis=1)
    except:
        fuzzy_unmatched_scopus['titles_plus_plus'] = fuzzy_unmatched_scopus.apply(lambda obs: '{} (VOL. {} ({}))'.format(obs.sc_title, obs.sc_vol, obs.sc_issue), axis=1)
    try: 
        fuzzy_unmatched_econlit['titles_plus_plus'] = fuzzy_unmatched_econlit.apply(lambda obs: '{} (VOL. {} ({}))'.format(obs.title, int(obs.volume), int(obs.issue)), axis=1)
    except:
        fuzzy_unmatched_econlit['titles_plus_plus'] = fuzzy_unmatched_econlit.apply(lambda obs: '{} (VOL. {} ({}))'.format(obs.title, obs.volume, obs.issue), axis=1)

    fuzzy_unmatched_scopus.sort_values(by=['sc_vol', 'sc_issue', 'sc_title'], inplace=True)
    fuzzy_unmatched_econlit.sort_values(by=['volume', 'issue', 'title'], inplace=True)

    post_fuzzy_scopus_titles = fuzzy_unmatched_scopus.titles_plus_plus.tolist()
    post_fuzzy_econlit_titles = fuzzy_unmatched_econlit.titles_plus_plus.tolist()

    
    lines = ['FUZZY MATCHING REPORT', bars, scopus_original, econlit_original, total_original, bars, naive_match, post_naive_scopus_only, post_naive_econlit_only, bars, fuzzy_match, post_fuzzy_scopus_only, post_fuzzy_econlit_only, fuzzy_unmatched_summary, bars]

    matching_report_path = 'econlit_scopus_matching_out/{}_fuzzy_results/{}_fuzzy_matching_report.txt'.format(pub_code, pub_code)
    with open(matching_report_path, 'w', encoding='utf-8') as report:
        report.write(str(datetime.now()))
        for line in lines:
            report.write('\n{}'.format(line))

        report.write('\nSCOPUS ARTICLES REMAINING UNMATCHED ({})\n'.format(len(fuzzy_unmatched_scopus)))
        for title in post_fuzzy_scopus_titles:
            report.write('\n\t{}'.format(title))



        report.write('\nECONLIT ARTICLES REMAINING UNMATCHED ({})\n'.format(len(fuzzy_unmatched_econlit)))
        for title in post_fuzzy_econlit_titles:
            report.write('\n\t{}'.format(title))

    return


In [69]:
run_list = [
    #### TOP 5 #####
    # 'AER',
    # 'ECA',
    # 'JPE',
    # 'QJE',
    # 'RES',
    #### IO & LAW AND ECON #####
    'ATB',
    # 'ALJ', IS NOT TRACKED BY ECONLIT
    # 'JEM',
    # 'JHR',
    # 'JLE',
    # 'JLO',
    # 'RJE',
    #### FINANCE #####
    # 'JOF',
    # 'JFE',
    # 'RFS',
    #### LABOR #####
    # 'JOL'
]

In [73]:
for pub_code in run_list:
    scopus_df, econlit_df = load_input_csvs(pub_code)
    print('----------------------------------------')
    # print_mode argument has options "none" (suppresses print statements), "removed" (prints observations that are deemed non-articles), "remaining" (prints articles remaining after removal)
    scopus_df, econlit_df = filter_non_articles(pub_code, scopus_df, econlit_df, print_mode="removed")
    naive_match_df = naive_match(scopus_df, econlit_df)
    print('----------------------------------------')
    nm_scopus_only, nm_econlit_only = left_right_onlys(naive_match_df)
    naive_match_df = naive_match_df[naive_match_df._merge == 'both']
    naive_match_df.reset_index(inplace=True)
    naive_match_df = naive_match_df.drop(columns=['_merge', 'level_0'], axis=1)
    print('----------------------------------------')
    unmatched_score_matrix = matrix_instantiation(nm_scopus_only, nm_econlit_only)
    print(unmatched_score_matrix.shape)
    matched_indices_list = index_matching(unmatched_score_matrix)
    interpret_matches(matched_indices_list, nm_scopus_only, nm_econlit_only)
    print('----------------------------------------')
    fuzzy_matches_appended = append_fuzzy_matches(matched_indices_list, nm_scopus_only, nm_econlit_only, naive_match_df)
    fuzzy_unmatched_scopus, fuzzy_unmatched_econlit = fuzzy_unmatched_remainders(matched_indices_list, nm_scopus_only, nm_econlit_only)
    print('----------------------------------------')
    output_fuzzy_matched_etc(pub_code, fuzzy_matches_appended, fuzzy_unmatched_scopus, fuzzy_unmatched_econlit)
    generate_matching_report(pub_code, scopus_df, econlit_df, naive_match_df, nm_scopus_only, nm_econlit_only, fuzzy_matches_appended, fuzzy_unmatched_scopus, fuzzy_unmatched_econlit)

READING IN ATB
ATB ------ NUMBER OF SCOPUS OBSERVATIONS: 770
ATB ----- NUMBER OF ECONLIT OBSERVATIONS: 989
----------------------------------------
ATB ------ SCOPUS NON-ARTICLES REMOVED: 45
ATB ----- ECONLIT NON-ARTICLES REMOVED: 99
Introduction
Introduction
Introduction
Organic or deliberate: A comment on "applying the ecosystem metaphor to entrepreneurship: Uses and abuses"
Response to long and Barr’s comments
Response to Parakkal’s comments
Response to Barr’s comments
Comment on William J. Curran III’s commitment and betrayal: Contradictions in American democracy, capitalism, and antitrust laws
Response to Parakkal’s comments
Markovits on defining monopolization: A comment
The economics of antitrust law: A comment on the other contributions to this symposium
The necessary complexity of predatory pricing analysis: A comment on richard s. markovits’s treatment of predatory pricing in economics and the interpretation and application of u.s. and e.u. antitrust law
Comments on richard m